In [ ]:
# на вход -> натренированная модель + последовательность днк (строка) -> на выходе предсказанная Hi-C (с возможностью сохранения)

In [6]:
# try to just load json here
# without python code => doesn't work because of custom layers.

# https://machinelearningmastery.com/save-load-keras-deep-learning-models/

import tensorflow as tf
from tensorflow.keras.models import load_model


# TODO: import code and load weights as usual
model = load_model("./test_size.h5")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


ValueError: Unknown layer: OneToTwo